In [ ]:
import numpy as np

import torch
torch.from_numpy(np.zeros(1))  # <- Forçage du lien numpy<->torch

import detectron2
import os
import json
import uuid
from PIL import Image
from tqdm import tqdm
import layoutparser as lp

# === CONFIG ===
IMAGE_DIR = "dataset_images"
ANNOTATED_DIR = "dataset_images_annotated"
COCO_JSON = "annotations_coco.json"
N_IMAGES = 5  # nombre d'images à annoter

os.makedirs(ANNOTATED_DIR, exist_ok=True)

# === LayoutParser Model ===
model = lp.Detectron2LayoutModel(
    config_path=r"C:\Users\DELL\bureau\bureau\VSCODE\model\config.yml",
    model_path=r"C:\Users\DELL\bureau\bureau\VSCODE\model\model_final.pth",
    label_map={
        0: "text",
        1: "title",
        2: "list",
        3: "table",
        4: "figure"
    },
    extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.6]
)

# === Mapping vers tes classes personnalisées ===
custom_map = {
    "title": "title",
    "text": "content",
    "figure": "media",
    "list": "sidebar",
    "table": "content"
}

# === Couleur différente pour chaque classe
def get_color(block):
    color_map = {
        "title": (255, 0, 0),     # Rouge
        "text": (0, 255, 0),      # Vert
        "figure": (0, 0, 255),    # Bleu
        "list": (255, 165, 0),    # Orange
        "table": (128, 0, 128),   # Violet
    }
    return color_map.get(block.type, (200, 200, 200))  # Gris par défaut

# === COCO structure ===
coco_output = {
    "images": [],
    "annotations": [],
    "categories": []
}

class_map = {name: idx + 1 for idx, name in enumerate(sorted(set(custom_map.values())))}
for name, idx in class_map.items():
    coco_output["categories"].append({"id": idx, "name": name})

annotation_id = 1
image_files = [f for f in os.listdir(IMAGE_DIR) if f.endswith(".jpg")][:N_IMAGES]

for img_id, filename in enumerate(tqdm(image_files, desc="Annotation")):
    image_path = os.path.join(IMAGE_DIR, filename)
    image = Image.open(image_path).convert("RGB")
    width, height = image.size

    # Ajouter image à COCO
    coco_output["images"].append({
        "id": img_id,
        "file_name": filename,
        "width": width,
        "height": height
    })

    layout = model.detect(image)
    from PIL import ImageDraw, ImageFont
    draw = image.copy()
    drawer = ImageDraw.Draw(draw)

# Facultatif : police pour le texte
    try:
          font = ImageFont.truetype("arial.ttf", size=14)
    except:
          font = ImageFont.load_default()

    for block in layout:
          label = block.type
          if label not in custom_map:
               continue

          color = get_color(block)
          x1, y1, x2, y2 = map(int, block.coordinates)
    
    # Rectangle
          drawer.rectangle([x1, y1, x2, y2], outline=color, width=2)
    
    # Texte
          drawer.text((x1 + 3, y1 + 3), label, fill=color, font=font)


    # 💡 Dessin avec couleur personnalisée et label
    
    draw.save(os.path.join(ANNOTATED_DIR, filename))

    for block in layout:
        label = block.type
        if label not in custom_map:
            continue

        coco_label = custom_map[label]
        x, y, x2, y2 = map(int, block.coordinates)
        w, h = x2 - x, y2 - y

        coco_output["annotations"].append({
            "id": annotation_id,
            "image_id": img_id,
            "category_id": class_map[coco_label],
            "bbox": [x, y, w, h],
            "area": w * h,
            "iscrowd": 0
        })
        annotation_id += 1

# === Save COCO JSON ===
with open(COCO_JSON, "w") as f:
    json.dump(coco_output, f, indent=2)

print(f"✅ Annotations enregistrées dans {COCO_JSON}")
print(f"🖼️ Images annotées sauvegardées dans : {ANNOTATED_DIR}")
